In [56]:
from google.colab import drive
drive.mount('/content/drive')

In [57]:
!git clone https://github.com/analokmaus/kuma_utils.git
import sys; sys.path.append("kuma_utils/")
import warnings; warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import joblib
from kuma_utils.preprocessing.imputer import LGBMImputer
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
train = pd.read_csv("drive/MyDrive/ML final/tabular-playground-series-aug-2022/train.csv")
test = pd.read_csv("drive/MyDrive/ML final/tabular-playground-series-aug-2022/test.csv")

In [58]:
y = train["failure"]
train = train.drop(columns=["id", "failure"])
test = test.drop(columns="id")
size = train.shape[0]
data = pd.concat((train, test))
data = pd.get_dummies(data)
train = data[:size]
test = data[size:]
index = train.columns
train_A = train[train["product_code_A"] == 1]
train_B = train[train["product_code_B"] == 1]
train_C = train[train["product_code_C"] == 1]
train_D = train[train["product_code_D"] == 1]
train_E = train[train["product_code_E"] == 1]
test_F = test[test["product_code_F"] == 1]
test_G = test[test["product_code_G"] == 1]
test_H = test[test["product_code_H"] == 1]
test_I = test[test["product_code_I"] == 1]

In [59]:
imputer = LGBMImputer(n_iter=100)
train_A = pd.DataFrame(data=imputer.fit_transform(train_A), columns=index)
train_B = pd.DataFrame(data=imputer.fit_transform(train_B), columns=index)
train_C = pd.DataFrame(data=imputer.fit_transform(train_C), columns=index)
train_D = pd.DataFrame(data=imputer.fit_transform(train_D), columns=index)
train_E = pd.DataFrame(data=imputer.fit_transform(train_E), columns=index)
test_F = pd.DataFrame(data=imputer.fit_transform(test_F), columns=index)
test_G = pd.DataFrame(data=imputer.fit_transform(test_G), columns=index)
test_H = pd.DataFrame(data=imputer.fit_transform(test_H), columns=index)
test_I = pd.DataFrame(data=imputer.fit_transform(test_I), columns=index)
train = pd.concat((train_A, train_B, train_C, train_D, train_E))
test = pd.concat((test_F, test_G, test_H, test_I))

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [60]:
column_name = ["product_code_A", "product_code_B", "product_code_C",
               "product_code_D", "product_code_E",
               "product_code_F", "product_code_G",
               "product_code_H", "product_code_I"]
train = train.drop(columns=column_name)
test = test.drop(columns=column_name)

In [61]:
pca = PCA(n_components=2)
train = pca.fit_transform(train)
test = pca.transform(test)

In [62]:
lr = LogisticRegression(n_jobs=-1)
param = {"C":[0.0001, 0.001, 0.01, 0.04, 0.05, 0.06, 0.07, 0.1],
         "max_iter":[100, 200, 500 ,1000],
         "solver":['lbfgs', 'liblinear', 'sag', 'saga'],
         "l1_ratio":[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
        }
model = RandomizedSearchCV(lr, param, verbose=3, cv=5)
model.fit(train, y)
joblib.dump(model, 'model')

['model1']